In [1]:
import findspark

findspark.init()

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
import time
import pandas
import joblib
import numpy as np

In [3]:
spark = SparkSession\
      .builder\
      .appName("Kafka_stm")\
      .getOrCreate()

In [4]:
df = spark\
     .readStream\
     .format("kafka")\
     .option("kafka.bootstrap.servers", "localhost:9092")\
     .option("subscribe", "waterQuality")\
     .option("startingOffsets", "earliest") \
     .option("maxOffsetsPerTrigger", 3600) \
     .option("stopGracefullyOnShutdown", "true") \
     .load()

df1 = df.selectExpr("CAST(value AS STRING)")

In [5]:
df1.printSchema()

df_void = StructType()\
	.add("tbVal", FloatType()) \
    .add("clVal", FloatType())\
	.add("fcltyMngNo", StringType())

df_sc = (df1.withColumn("value", from_json("value",df_void)))

print(type(df))
df_sc.printSchema()

root
 |-- value: string (nullable = true)

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- value: struct (nullable = true)
 |    |-- tbVal: float (nullable = true)
 |    |-- clVal: float (nullable = true)
 |    |-- fcltyMngNo: string (nullable = true)



In [16]:


df2 = df1.select(from_json(col("value"), df_void).alias("df"))
df3 = df2.select("df.*")

df3.printSchema()

# feature_columns = df3.columns[:1]
# assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

# df4 = assembler.transform(df3)


root
 |-- tbVal: float (nullable = true)
 |-- clVal: float (nullable = true)
 |-- fcltyMngNo: string (nullable = true)



In [6]:
query = df_sc.writeStream.format("console").outputMode("Append").start()
time.sleep(3600) # sleep 10 seconds
query.stop()

KeyboardInterrupt: 

In [7]:
import joblib
import numpy as np

# Load the model from the file
loaded_model = joblib.load('./water_test4')

/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own r

In [1]:
# Create a numpy array containing a new observation (for example tomorrow's seasonal and weather forecast information)
X_new = np.array(df_sc).astype(list)
#.astype('float64')
# print ('New sample: {}'.format(list(X_new[0])))

# Use the model to predict tomorrow's rentals
result = loaded_model.predict(X_new)
print('Prediction: {0} Ph'.format(np.round(result[0])))

NameError: name 'np' is not defined